# Dataset and preprocessing

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
import os

class DoseDataset(Dataset):
    def __init__(self, image_paths, root,transfo=transforms.ToTensor(),test=False):
        self.image_paths = image_paths
        self.transform = transfo
        self.root = root
        self.test = test

        
    def __getitem__(self, index):
        ct_path = self.root + self.image_paths[index] + '/ct.npy'
        dose = torch.zeros(1)
        if self.test == False:
            dose_path = self.root + self.image_paths[index] + '/dose.npy'
        possible_dose_mask_path = self.root + self.image_paths[index] + '/possible_dose_mask.npy'
        structure_masks_path = self.root + self.image_paths[index] + '/structure_masks.npy'

        ct = np.expand_dims(np.load(ct_path),0)
        if self.test == False:
            dose = np.expand_dims(np.load(dose_path),0)
            dose = torch.Tensor(dose)

        possible_dose_mask = np.expand_dims(np.load(possible_dose_mask_path),0)
        structure_masks = np.load(structure_masks_path)
        
        epsillon = 1e-7
        ct = (ct - ct.min())/(ct.max()-ct.min()+epsillon)
        x = np.concatenate((ct,structure_masks,possible_dose_mask))
        x = self.transform(x)
        
        if self.test == True:
            return x,possible_dose_mask,self.image_paths[index]
        return x,possible_dose_mask,dose
    
    def __len__(self):
        return len(self.image_paths)


rootTrain = '/kaggle/input/datasetdose/train/train/'
paths = os.listdir(rootTrain)

datasetTrain = DoseDataset(paths,rootTrain,transfo=transforms.ToTensor())


batch_size = 32
trainLoader = torch.utils.data.DataLoader(
    datasetTrain,
    batch_size=batch_size, shuffle=True, num_workers=1)

rootVal = '/kaggle/input/datasetdose/validation/validation/'
pathsVal = os.listdir(rootVal)
datasetVal = DoseDataset(pathsVal,rootVal,transfo=transforms.ToTensor())

valLoader = torch.utils.data.DataLoader(
    datasetVal,
    batch_size=batch_size, shuffle=True, num_workers=1)



rootTest = '/kaggle/input/datasetdose/test/test/'
pathsTest = os.listdir(rootTest)
datasetTest = DoseDataset(pathsTest,rootTest ,transfo=transforms.ToTensor(),test=True)

testLoader = torch.utils.data.DataLoader(
    datasetTest,
    batch_size=batch_size, shuffle=True, num_workers=1)

# First model: U-net with encoder fully depthwise convolutional

In [ ]:
import torch.nn as nn

class Block(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size,nbGp=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size, padding='same',groups=nbGp)
        self.relu  = nn.LeakyReLU(negative_slope=0.03)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size, padding='same',groups=nbGp)
        self.bn = nn.BatchNorm2d(out_ch)#nn.GroupNorm(4, out_ch)#nn.Identity()#nn.BatchNorm2d(out_ch)
    
    def forward(self, x):
        return self.conv2(self.relu(self.bn(self.conv1(x))))
    
class Encoder(nn.Module):
    def __init__(self, chs, kernel_size):
        super().__init__()
        self.enc_blocks = nn.ModuleList([Block(chs[i], chs[i+1], kernel_size,nbGp = chs[i]) for i in range(len(chs)-1)])
        self.pool       = nn.MaxPool2d(2)
    
    def forward(self, x):
        ftrs = []
        for block in self.enc_blocks:
            x = block(x)
            ftrs.append(x)
            x = self.pool(x)
        return ftrs

class DecoderTranspose(nn.Module):
    def __init__(self, chs, kernel_size):
        super().__init__()
        self.chs         = chs
        self.upconvs    = nn.ModuleList([nn.ConvTranspose2d(chs[i], chs[i+1], 2, 2) for i in range(len(chs)-1)]) ## Convolution transposée
        self.dec_blocks  = nn.ModuleList([Block(chs[i], chs[i+1], kernel_size) for i in range(len(chs)-1)]) 
        
    def forward(self, x, encoder_features):
        for i in range(len(self.chs)-1):
            x        = self.upconvs[i](x) 
            enc_ftrs = self.crop(encoder_features[i], x)
            x        = torch.cat([x, enc_ftrs], dim=1)
            x        = self.dec_blocks[i](x)
        return x
    
    def crop(self, enc_ftrs, x):
        _, _, H, W = x.shape
        enc_ftrs   = torchvision.transforms.CenterCrop([H, W])(enc_ftrs)
        return enc_ftrs
    

In [ ]:
!pip install pytorch_model_summary

In [ ]:
from pytorch_model_summary import summary
import torchvision
class UNetConvTranspose(nn.Module):
    def __init__(self, channels, kernel_size):
        super().__init__()
        self.encoder     = Encoder((12,) + channels, kernel_size)
        self.decoder     = DecoderTranspose(tuple(reversed(channels)), kernel_size)
        self.head        = nn.Conv2d(channels[0], 1, 1)

    def forward(self, x):
        enc_ftrs = self.encoder(x)
        out      = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        out      = self.head(out)
        return out


# Testing dimensions
sig = torch.normal(0,1,(1,12,128,128))
test_model = UNetConvTranspose(kernel_size=3, channels=(24,48,48*2,48*4,48*8)).cpu()
test_tensor = sig.cpu()
_ = summary(test_model, test_tensor, print_summary=True, show_input=False, max_depth=2, show_parent_layers=True)
del test_model
del test_tensor

-----------------------------------------------------------------------------------------------------------------
                        Parent Layers          Layer (type)          Output Shape         Param #     Tr. Param #
            UNetConvTranspose/Encoder               Block-1     [1, 24, 128, 128]             744             744
            UNetConvTranspose/Encoder           MaxPool2d-2       [1, 24, 64, 64]               0               0
            UNetConvTranspose/Encoder               Block-3       [1, 48, 64, 64]           1,488           1,488
            UNetConvTranspose/Encoder               Block-4       [1, 96, 32, 32]           2,976           2,976
            UNetConvTranspose/Encoder               Block-5      [1, 192, 16, 16]           5,952           5,952
            UNetConvTranspose/Encoder               Block-6        [1, 384, 8, 8]          11,904          11,904
   UNetConvTranspose/DecoderTranspose     ConvTranspose2d-7      [1, 192, 16, 16]       

In [ ]:

def train(epoch, network,trainLoss,train_dataloader,optimizer):
    network.train()
    lossEpoch = []
    for i,(batch,mask,labs) in enumerate(train_dataloader):
        batch = batch.permute(0,2,1,3)

        batch = batch.to(device)
        labs = labs.to(device)
        mask = mask.to(device)
            
        optimizer.zero_grad()
        output = network(batch.float())*mask
        lossFun = torch.nn.L1Loss()
        loss = lossFun(output, labs.long())
        lossEpoch.append(loss.item())
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(batch), len(train_dataloader.dataset),
                100. * i / len(train_dataloader), loss.item()))
    print('Mean loss of the epoch: ',np.mean(np.array(lossEpoch)))
    trainLoss.append(np.mean(np.array(lossEpoch)))


def test(network,testLoss,test_dataloader,device,batch_size=32):
    network.eval()
    test_loss = 0
    correct = 0
    for data, mask,target in test_dataloader:
        data = data.permute(0,2,1,3)

        data=data.to(device)
        target=target.to(device)
        mask = mask.to(device)

        output = network(data.float())*mask

        lossFun = torch.nn.L1Loss()
        test_loss += lossFun(output, target.long()).item() # sum up batch loss
        pred = output.data

    test_loss /= len(test_dataloader.dataset)
    testLoss.append(test_loss*batch_size)
    print('\nTest set: Average loss: {:.4f}\n'.format(
        test_loss*batch_size))
    return test_loss

In [ ]:
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
best_loss = float('inf')
model = UNetConvTranspose(kernel_size=3, channels=(24,24*2,24*4,24*8))
optimizer = torch.optim.Adam(model.parameters(),lr=3e-4,weight_decay=1e-5)
 
model.train()
model=model.to(device)
n_epochs = 30
trainLoss = []
testLoss = []
for k in range(n_epochs):
    train(k,model,trainLoss,trainLoader,optimizer)
    b2 = test(model,testLoss,valLoader,device,batch_size)
    if b2 < best_loss:
        best_loss = b2
        torch.save(model.state_dict(), './model'+str(k)+'.pt')

cuda
Train Epoch: 0 [0/7800 (0%)]	Loss: 2.645636
Train Epoch: 0 [3200/7800 (41%)]	Loss: 0.789058
Train Epoch: 0 [6400/7800 (82%)]	Loss: 0.882802
Mean loss of the epoch:  1.1103845896812086

Test set: Average loss: 0.7654

Train Epoch: 1 [0/7800 (0%)]	Loss: 0.797189
Train Epoch: 1 [3200/7800 (41%)]	Loss: 0.737379
Train Epoch: 1 [6400/7800 (82%)]	Loss: 0.806788
Mean loss of the epoch:  0.7178150366073568

Test set: Average loss: 0.6915

Train Epoch: 2 [0/7800 (0%)]	Loss: 0.725914
Train Epoch: 2 [3200/7800 (41%)]	Loss: 0.577207
Train Epoch: 2 [6400/7800 (82%)]	Loss: 0.715865
Mean loss of the epoch:  0.6470969121202774

Test set: Average loss: 0.6847

Train Epoch: 3 [0/7800 (0%)]	Loss: 0.553058
Train Epoch: 3 [3200/7800 (41%)]	Loss: 0.626164
Train Epoch: 3 [6400/7800 (82%)]	Loss: 0.585732


KeyboardInterrupt: 

## ImageNet Pretrained model

In [ ]:
!pip install segmentation_models_pytorch

In [ ]:

import segmentation_models_pytorch as smp
model = smp.DeepLabV3Plus('resnet50', classes=1, in_channels=12)


from torch import nn
model.train()
model=model.to(device)
n_epochs = 100
trainLoss = []
testLoss = []
optimizer = torch.optim.Adam(model.parameters(),lr=1e-2,weight_decay=2e-5)
for k in range(n_epochs):
    train(k,model,trainLoss,trainLoader,optimizer)
    b2 = test(model,testLoss,testLoader,device,batch_size)
    if b2 < best_loss:
        best_loss = b2
        torch.save(model.state_dict(), './model'+str(k)+'.pt')

## Brain segmentation pretrained models

In [ ]:
import torch
import torch.nn as nn
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32,pretrained=True)


Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [ ]:
model

UNet(
  (encoder1): Sequential(
    (enc1conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu1): ReLU(inplace=True)
    (enc1conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu2): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder2): Sequential(
    (enc2conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc2relu1): ReLU(inplace=True)
    (enc2conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [ ]:
#We define a new forward to not have the sigmoid in it
def new_forward(self, x):
    enc1 = self.encoder1(x)
    enc2 = self.encoder2(self.pool1(enc1))
    enc3 = self.encoder3(self.pool2(enc2))
    enc4 = self.encoder4(self.pool3(enc3))

    bottleneck = self.bottleneck(self.pool4(enc4))

    dec4 = self.upconv4(bottleneck)

    dec4 = torch.cat((dec4, enc4), dim=1)
    dec4 = self.decoder4(dec4)
    dec3 = self.upconv3(dec4)
    dec3 = torch.cat((dec3, enc3), dim=1)
    dec3 = self.decoder3(dec3)
    dec2 = self.upconv2(dec3)
    dec2 = torch.cat((dec2, enc2), dim=1)
    dec2 = self.decoder2(dec2)
    dec1 = self.upconv1(dec2)
    dec1 = torch.cat((dec1, enc1), dim=1)
    dec1 = self.decoder1(dec1)
    return self.conv(dec1)


bound_method = new_forward.__get__(model, model.__class__)
setattr(model, 'forward', bound_method)


In [ ]:
model.encoder1.enc1conv1 = torch.nn.Conv2d(12, 36,kernel_size=(3,3),groups = 12,padding='same')
model.encoder1.enc1norm1 = torch.nn.BatchNorm2d(36)
model.encoder1.enc1conv2 = torch.nn.Conv2d(36, 32,kernel_size=(3,3),padding='same')


model.decoder1.dec1conv1 = torch.nn.Conv2d(64, 36,kernel_size=(3,3),padding='same')
model.decoder1.dec1norm1 =  torch.nn.BatchNorm2d(36)
model.decoder1.dec1conv2 = torch.nn.Conv2d(36, 32,kernel_size=(3,3),padding='same')


In [ ]:
for param in model.parameters():
    param.requires_grad = False

for param in model.encoder1.parameters():
    param.requires_grad = True

for param in model.decoder1.parameters():
    param.requires_grad = True

In [ ]:
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
best_loss = float('inf')
optimizer = torch.optim.Adam(model.parameters(),lr=3e-4,weight_decay=2e-5)

model.train()
model=model.to(device)
n_epochs = 35
trainLoss = []
testLoss = []
for k in range(n_epochs):
    train(k,model,trainLoss,trainLoader,optimizer)
    b2 = test(model,testLoss,valLoader,device,batch_size)
    if b2 < best_loss:
        best_loss = b2
        torch.save(model.state_dict(), './model'+str(k)+'.pt')
    if k == 5:
        for param in model.parameters():
          param.requires_grad = False


# Training from scratch

In [ ]:
import torch
import torch.nn as nn
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32,pretrained=True)


Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [ ]:
#We define a new forward to not have the sigmoid in it
def new_forward(self, x):
    enc1 = self.encoder1(x)
    enc2 = self.encoder2(self.pool1(enc1))
    enc3 = self.encoder3(self.pool2(enc2))
    enc4 = self.encoder4(self.pool3(enc3))

    bottleneck = self.bottleneck(self.pool4(enc4))

    dec4 = self.upconv4(bottleneck)

    dec4 = torch.cat((dec4, enc4), dim=1)
    dec4 = self.decoder4(dec4)
    dec3 = self.upconv3(dec4)
    dec3 = torch.cat((dec3, enc3), dim=1)
    dec3 = self.decoder3(dec3)
    dec2 = self.upconv2(dec3)
    dec2 = torch.cat((dec2, enc2), dim=1)
    dec2 = self.decoder2(dec2)
    dec1 = self.upconv1(dec2)
    dec1 = torch.cat((dec1, enc1), dim=1)
    dec1 = self.decoder1(dec1)
    return self.conv(dec1)


bound_method = new_forward.__get__(model, model.__class__)
setattr(model, 'forward', bound_method)


In [ ]:
model.encoder1.enc1conv1 = torch.nn.Conv2d(12, 36,kernel_size=(3,3),groups = 12,padding='same')
model.encoder1.enc1norm1 = torch.nn.BatchNorm2d(36)
model.encoder1.enc1conv2 = torch.nn.Conv2d(36, 32,kernel_size=(3,3),padding='same')


model.decoder1.dec1conv1 = torch.nn.Conv2d(64, 36,kernel_size=(3,3),padding='same')
model.decoder1.dec1norm1 =  torch.nn.BatchNorm2d(36)
model.decoder1.dec1conv2 = torch.nn.Conv2d(36, 32,kernel_size=(3,3),padding='same')


In [ ]:
import numpy as np
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
best_loss = float('inf')
optimizer = torch.optim.AdamW(model.parameters(),lr=2e-4,weight_decay=1e-4)

model.train()
model=model.to(device)
n_epochs = 35
trainLoss = []
testLoss = []
for k in range(n_epochs):
    train(k,model,trainLoss,trainLoader,optimizer)
    b2 = test(model,testLoss,valLoader,device,batch_size)
    if b2 < best_loss:
        best_loss = b2
        torch.save(model.state_dict(), './model'+str(k)+'.pt')


cuda


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f086a7fd8c0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Train Epoch: 0 [0/7800 (0%)]	Loss: 2.968388
Train Epoch: 0 [3200/7800 (41%)]	Loss: 2.198259
Train Epoch: 0 [6400/7800 (82%)]	Loss: 2.349297
Mean loss of the epoch:  2.3748356985156462


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f086a7fd8c0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process



Test set: Average loss: 2.0870

Train Epoch: 1 [0/7800 (0%)]	Loss: 2.644021
Train Epoch: 1 [3200/7800 (41%)]	Loss: 1.971743
Train Epoch: 1 [6400/7800 (82%)]	Loss: 1.936213
Mean loss of the epoch:  2.0392513751041683

Test set: Average loss: 1.9865

Train Epoch: 2 [0/7800 (0%)]	Loss: 1.879202
Train Epoch: 2 [3200/7800 (41%)]	Loss: 1.484448
Train Epoch: 2 [6400/7800 (82%)]	Loss: 1.422986
Mean loss of the epoch:  1.7073008115310222

Test set: Average loss: 1.5672

Train Epoch: 3 [0/7800 (0%)]	Loss: 1.344557
Train Epoch: 3 [3200/7800 (41%)]	Loss: 1.396452
Train Epoch: 3 [6400/7800 (82%)]	Loss: 1.482971
Mean loss of the epoch:  1.3657783982167389

Test set: Average loss: 1.0515

Train Epoch: 4 [0/7800 (0%)]	Loss: 1.455244
Train Epoch: 4 [3200/7800 (41%)]	Loss: 1.148333
Train Epoch: 4 [6400/7800 (82%)]	Loss: 0.958011
Mean loss of the epoch:  1.027883668844896

Test set: Average loss: 0.8548

Train Epoch: 5 [0/7800 (0%)]	Loss: 0.793739
Train Epoch: 5 [3200/7800 (41%)]	Loss: 0.425741
Train Ep

KeyboardInterrupt: 

In [ ]:
!mkdir /kaggle/working/evaluation
def evaluateCodaLab(model,testData):
    model.eval()
    
    for k,(x,m,s) in enumerate(testData):
        m = m.to(device)
        x = x.permute(0,2,1,3).float().to(device)

        with torch.no_grad():
            pred = model(x)*m
        pred = np.array(pred.cpu())
        print(s)
        for i in range(pred.shape[0]):
            np.save('/kaggle/working/evaluation/'+s[i]+'.npy',pred[i])
        
            

mkdir: cannot create directory ‘/kaggle/working/evaluation’: File exists


In [ ]:
evaluateCodaLab(model,testLoader)

('sample_9603', 'sample_9852', 'sample_9966', 'sample_9631', 'sample_9975', 'sample_10120', 'sample_9696', 'sample_9472', 'sample_10129', 'sample_9429', 'sample_9000', 'sample_9167', 'sample_9864', 'sample_9051', 'sample_10199', 'sample_9936', 'sample_9453', 'sample_9798', 'sample_9523', 'sample_9835', 'sample_9964', 'sample_9969', 'sample_9379', 'sample_9329', 'sample_9282', 'sample_9571', 'sample_9945', 'sample_9815', 'sample_9685', 'sample_9736', 'sample_10184', 'sample_9691')
('sample_10140', 'sample_9942', 'sample_9020', 'sample_9656', 'sample_9667', 'sample_9884', 'sample_10177', 'sample_9632', 'sample_9644', 'sample_9730', 'sample_9223', 'sample_9607', 'sample_9510', 'sample_10005', 'sample_9342', 'sample_10128', 'sample_9971', 'sample_10100', 'sample_9266', 'sample_9550', 'sample_9414', 'sample_9700', 'sample_10102', 'sample_9294', 'sample_9989', 'sample_9802', 'sample_9045', 'sample_9525', 'sample_9055', 'sample_9306', 'sample_9466', 'sample_9161')
('sample_9244', 'sample_9408

In [ ]:
import shutil
shutil.make_archive('submission', 'zip', '/kaggle/working/evaluation')

'/kaggle/working/submission.zip'